In [1]:
import random
import math

# ------- Distance Function -------
def path_length(path, dist_matrix):
    total = 0
    for i in range(len(path)):
        total += dist_matrix[path[i]][path[(i+1) % len(path)]]
    return total

# ------- Generate a random tour -------
def random_tour(n):
    tour = list(range(n))
    random.shuffle(tour)
    return tour

# ------- Levy Flight (swap mutation) -------
def levy_flight(path):
    new_path = path[:]
    i, j = random.sample(range(len(path)), 2)
    new_path[i], new_path[j] = new_path[j], new_path[i]
    return new_path

# ------- Cuckoo Search Algorithm -------
def cuckoo_search(dist_matrix, n_nests=15, pa=0.25, max_iter=100):

    n = len(dist_matrix)
    nests = [random_tour(n) for _ in range(n_nests)]
    fitness = [path_length(nest, dist_matrix) for nest in nests]

    best = nests[fitness.index(min(fitness))]

    print("Initial best length:", path_length(best, dist_matrix))

    for it in range(1, max_iter + 1):

        # Generate a new solution via Levy Flight
        cuckoo = levy_flight(best)
        cuckoo_fit = path_length(cuckoo, dist_matrix)

        # Replace a random nest if better
        k = random.randrange(n_nests)
        if cuckoo_fit < fitness[k]:
            nests[k] = cuckoo
            fitness[k] = cuckoo_fit

        # Abandon worst nests with probability pa
        for i in range(n_nests):
            if random.random() < pa:
                nests[i] = random_tour(n)
                fitness[i] = path_length(nests[i], dist_matrix)

        # Update best
        current_best = nests[fitness.index(min(fitness))]
        if path_length(current_best, dist_matrix) < path_length(best, dist_matrix):
            best = current_best

        # Print required checkpoints
        if it == 50:
            print("Iteration 50:", path_length(best, dist_matrix))
        if it == 100:
            print("Iteration 100:", path_length(best, dist_matrix))

    return best, path_length(best, dist_matrix)


# ------- Example TSP Distance Matrix -------
# 10-city symmetric matrix (example)
dist_matrix = [
    [0,29,20,21,16,31,100,12,4,31],
    [29,0,15,29,28,40,72,21,29,41],
    [20,15,0,15,14,25,81,9,23,27],
    [21,29,15,0,4,12,92,12,25,13],
    [16,28,14,4,0,16,94,9,20,16],
    [31,40,25,12,16,0,95,24,36,3],
    [100,72,81,92,94,95,0,90,101,99],
    [12,21,9,12,9,24,90,0,15,25],
    [4,29,23,25,20,36,101,15,0,35],
    [31,41,27,13,16,3,99,25,35,0]
]

# ------- Run the Cuckoo Search -------
best_path, best_len = cuckoo_search(dist_matrix)


Initial best length: 318
Iteration 50: 258
Iteration 100: 249
